In [1]:
import os
import random
import pandas as pd
from datetime import datetime
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6


# Dataset Loading

In [2]:
movies = pd.read_csv("./datasets/movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [3]:
titles = movies['title'].tolist()
random.shuffle(titles)
print("Movies in the Dataset:", len(titles))

Movies in the Dataset: 62423


# LLama2 Set-Up

In [4]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [6]:
# Loading the LLM on GPU
llm = Llama(
    model_path=model_path,
    n_threads=16,
    n_batch=768,
    n_gpu_layers=-1,
    verbose=False
)

llama.cpp: loading model from /home/xodiec/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_inte

In [7]:
dictionary = dict()

In [8]:
i = 0
for title in titles:
    start = datetime.now()
    prompt = f"I would like you to tell me the following informations related to the movie {title}: Genre, Brief description of the plot, Director, Principal 4/6 members of the cast."
    prompt_template = f'''SYSTEM: You are a helpful assistant who's expert in the field of Cinema. For the requested movie, strictly answer as follows:
        
        [START]
        - Movie Title
        - Genre: use as least tokens as possible for this information
        - Description of the plot: report the main details of the movie's plot
        - Director: only report the director's name and surname
        - Principal 4/6 Members of the Cast: report names and surnames
        [END]

        The '[START]' token must be the first token you generate; after the '[START]' token there suddenly must be the movie's information list I requested.
        After the list there must be the '[END]' token, which must be the last token you generate.

        USER: {prompt}

        ASSISTANT:
        '''

    response = llm(
        prompt=prompt_template,
        max_tokens=512,
        temperature=0.1,
        top_p=0.85,
        repeat_penalty=1.1,
        top_k=85,
        echo=False
    )

    end = datetime.now()
    print(f"PROCESSED {title}, required time: {str(end-start)}")
    # print(response["choices"][0]["text"])
    # print()
    dictionary[title] = response["choices"][0]["text"]

    i+=1
    if i == 20:
        break

PROCESSED Double Jeopardy (1955), required time: 0:00:08.839278
PROCESSED Sicko (2007), required time: 0:00:09.709252
PROCESSED Dead Air (2009), required time: 0:00:08.556086
PROCESSED David and Goliath (2015), required time: 0:00:10.301520
PROCESSED Species: The Awakening (2007), required time: 0:00:12.081652
PROCESSED Locked In (2010), required time: 0:00:10.422119
PROCESSED Red Sun Rising (1994), required time: 0:00:11.199476
PROCESSED Criss Cross (1949), required time: 0:00:08.379841
PROCESSED Tomorrow, on April 3rd... (1969), required time: 0:00:13.137004
PROCESSED Return of Shanghai Joe (1975), required time: 0:00:08.377267
PROCESSED London Heist (2017), required time: 0:00:08.001636
PROCESSED Scream 3 (2000), required time: 0:00:11.811106
PROCESSED Inspector Palmu's Error (Komisario Palmun erehdys) (1960), required time: 0:00:12.340030
PROCESSED Love on a Leash (2011), required time: 0:00:12.276547
PROCESSED War Is Hell (1963), required time: 0:00:09.417609
PROCESSED The Audienc

In [13]:
dictionary["Dead Air (2009)"]

'\n        [START]\n        - Movie Title: Dead Air\n        - Genre: Horror\n        - Description of the plot: The movie follows a group of friends who are terrorized by a mysterious entity while broadcasting their college radio show.\n        - Director: Jesse Holland and Andy Mitton\n        - Principal 4/6 Members of the Cast:\n            - Michael Bowen as Sheriff Barker\n            - Jesse Holland as Danny\n            - Andy Mitton as Oliver\n            - Kandace Caine as Lily\n        [END]'